In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
!pip install datasets
import datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
gender_dataset = datasets.load_dataset("AudioLLMs/iemocap_gender_recognition")["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/140M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
def extract_gender(example):
    if "female" in example["answer"].lower():
        example["answer"] = 1
    else:
        example["answer"] = 0
    return example

In [ ]:
gender_dataset = gender_dataset.map(extract_gender)

Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
gender_dataset = gender_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
from transformers import AutoFeatureExtractor

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["context"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
encoded_gender_dataset = gender_dataset.map(preprocess_function, remove_columns="context", batched=True)
encoded_gender_dataset = encoded_gender_dataset.rename_column("answer", "label")
encoded_gender_dataset = encoded_gender_dataset.remove_columns(["instruction"])

Map:   0%|          | 0/803 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
encoded_gender_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 803
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 201
    })
})

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=2
).to(device)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
training_args = TrainingArguments(
    output_dir="gender_classifier_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    report_to = "none",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    disable_tqdm = False,
    no_cuda = False,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_gender_dataset["train"],
    eval_dataset=encoded_gender_dataset["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689335,0.592040
2,0.656500,0.673798,0.691542
3,0.613300,0.588361,0.800995
4,0.613300,0.432113,0.855721
5,0.451400,0.378767,0.875622
6,0.368800,0.367912,0.855721
7,0.368800,0.300462,0.905473
8,0.303900,0.300758,0.905473


TrainOutput(global_step=60, training_loss=0.45529173215230306, metrics={'train_runtime': 421.2735, 'train_samples_per_second': 19.061, 'train_steps_per_second': 0.142, 'total_flos': 6.296939599104e+16, 'train_loss': 0.45529173215230306, 'epoch': 8.615384615384615})

In [ ]:
trainer.save_model("./gender_classification")

In [ ]:
model = AutoModelForAudioClassification.from_pretrained(
    "nikzagl/wav2vec-finetuned-on-gender-classification", num_labels=2
).to(device)

config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("nikzagl/wav2vec-finetuned-on-gender-classification")

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]